In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV_small.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [2]:
ratings = df
ratings = ratings.drop("helpful")
ratings = ratings.drop("reviewText")
ratings = ratings.drop("reviewTime")
ratings = ratings.drop("reviewerName")
ratings = ratings.drop("summary")
ratings = ratings.drop("unixReviewTime")
ratings.columns

['asin', 'overall', 'reviewerID']

In [31]:
users = ratings.map(lambda r: r.reviewerID).distinct().zipWithIndex()
users.take(5)
users.count()

307105

In [32]:
movies = ratings.map(lambda r: r.asin).distinct().zipWithIndex()
movies.take(5)
movies.count()

9852

In [5]:
moviesDf = sqlContext.createDataFrame(movies, ['asin', 'movieID'])
moviesDf.take(5)
usersDf = sqlContext.createDataFrame(users, ['reviewerID', 'userID'])
usersDf.take(5)

[Row(reviewerID=u'A1PZF2ZLAG4DCX', userID=130924),
 Row(reviewerID=u'A137LELERBZDUO', userID=130925),
 Row(reviewerID=u'A37S4Y1K41C91M', userID=130926),
 Row(reviewerID=u'A1YIFJMBYD30HP', userID=130927),
 Row(reviewerID=u'A21TGXU3IAAZYG', userID=130928)]

In [6]:
rawRatings1 = ratings.join(usersDf, ratings.reviewerID == usersDf.reviewerID).drop("reviewerID")

In [7]:
rawRatings2 = rawRatings1.join(moviesDf, ratings.asin == moviesDf.asin).drop("asin")

In [9]:
rawRatings3 = rawRatings2.map(lambda row: (row.userID, row.movieID, row.overall))

In [10]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [11]:
rank = 10
numIterations = 10
model = ALS.train(rawRatings3, rank, numIterations)

In [41]:
userMoviePair = usersDf.join(moviesDf).drop('reviewerID').drop('asin')
userMoviePair.printSchema()
#userMoviePair = userMoviePair.filter(userMoviePair.userID < 10000)
userMoviePair = userMoviePair.map(lambda row: (row.userID, row.movieID))

root
 |-- userID: long (nullable = true)
 |-- movieID: long (nullable = true)



In [44]:
recommendations = model.predictAll(userMoviePair)

In [45]:
recommendationsToC = recommendations.map(lambda x: {"user":x[0], "product":x[1], "rating":x[2]})

In [51]:
def AddToCassandra_allcountsbatch_bypartition(d_iter):
        from cqlengine import columns
        from cqlengine.models import Model
        from cqlengine import connection
        from cqlengine.management import sync_table
        CASSANDRA_KEYSPACE = "playground"
        class reviewerProfile(Model):
                user = columns.Integer(primary_key=True)
                product = columns.Integer(primary_key=True)
                rating = columns.Float(primary_key=True, clustering_order="DESC")
                
        connection.setup(['172.31.39.226'], CASSANDRA_KEYSPACE)
        sync_table(reviewerProfile)
        for d in d_iter:
                reviewerProfile.create(**d)

# Create table if it does not exist. Need to do this before submitting to Spark to avoid collisions
AddToCassandra_allcountsbatch_bypartition([])
recommendationsToC.foreachPartition(AddToCassandra_allcountsbatch_bypartition)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job 46 cancelled because Stage 529 was cancelled
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1273)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1225)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1213)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1212)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1212)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:156)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:1